# 추천 시스템의 유형을 적으시오.   
1. 콘텐츠 기반 필터링(Content based filtering)
2. 협업 필터링(Collaborative filtering)
    * 최근접 이웃(Nearest Neighbor)
    * 잠재 요인(Latent Factor)

# 콘텐츠 기반 필터링 방식이란?
사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식.

# 협업 필터링방식이란?
사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행하는 것.

# 협업필터링의 주요 목표는?
사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가 하는 것.

# 최근접 이웃 협업 필터의 두가지 방식에 대하여 적으시오
1. 사용자 기반(User-User): 특정 사용자와 유사한 다른 사용자를 TOP-N으로 선정해 이 TOP-N 사용자가 좋아하는 아이템을 추천하는 방식.
2. 아이템 기반(Item-Item): 아이템이 가지는 속성과는 상관없이 사용자들이 그 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템을 추천하는 방식.

# 잠재 요인 협업 필터링이란?
사용자-아이템 평점 매트릭스 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법.

# 잠재 요인 협업 필터링이 사용하는 기법에 대해 적으시오
행렬 분해를 이용.

# 행렬 분해를 이용한 잠재 요인 협업 필터링 실습 문제

In [2]:
# Library import
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
import pandas as pd
from surprise import Reader, Dataset

## 데이터 준비

In [9]:
# 데이터 출처: https://www.kaggle.com/sengzhaotoo/movielens-small?select=movies.csv  
rating = pd.read_csv('ratings.csv')
movie = pd.read_csv('movies.csv')

In [4]:
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
movie.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## 데이터 전처리

rating에서 timestamp drop하기

In [10]:
rating.drop('timestamp', axis=1, inplace=True)

In [11]:
rating.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


movie에서 genres drop하기

In [13]:
movie.drop('genres', axis=1, inplace=True)
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [25]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [17]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [23]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix
# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')


In [26]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9320565818473825
### iteration step :  10  rmse :  0.7499900285865972
### iteration step :  20  rmse :  0.5275349643538034
### iteration step :  30  rmse :  0.38354378256464045
### iteration step :  40  rmse :  0.3017115996487719
### iteration step :  50  rmse :  0.2547302025929975
### iteration step :  60  rmse :  0.22614838269743007
### iteration step :  70  rmse :  0.20743903539394296
### iteration step :  80  rmse :  0.19434445700230898
### iteration step :  90  rmse :  0.18468747401810467
### iteration step :  100  rmse :  0.17727940797831152
### iteration step :  110  rmse :  0.17142224690422145
### iteration step :  120  rmse :  0.16667903477143253
### iteration step :  130  rmse :  0.16276190961946319
### iteration step :  140  rmse :  0.1594735259015614
### iteration step :  150  rmse :  0.15667423337324488
### iteration step :  160  rmse :  0.15426258592092004
### iteration step :  170  rmse :  0.1521632200876546
### iteration step :  180  r

In [27]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,1.641676,2.889387,1.099882,0.295160,1.345731,1.774250,1.729358,2.561874,2.762497,2.718208,...,2.289714,0.664554,2.501499,1.309686,2.337709,1.995150,0.530999,2.579704,2.272978,2.022199
2,2.164730,3.624265,1.494340,0.269798,1.933644,2.503397,2.519713,2.729019,3.605812,2.809703,...,3.298091,0.770810,2.924901,1.390264,2.935596,3.271624,0.628967,2.769774,2.934985,2.792867
3,2.142794,3.537876,1.403330,0.337382,1.591148,2.406600,2.453896,3.081602,3.818645,3.110110,...,3.323351,1.038857,3.539153,3.361631,3.097248,1.265515,0.598941,2.450413,3.032541,2.731907


In [28]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [29]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [30]:
# 29번 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 29)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 29, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies


,pred_score
title,
Starship Troopers (1997),5.390790
American History X (1998),5.325811
Little Miss Sunshine (2006),5.299548
"Prestige, The (2006)",5.255162
"English Patient, The (1996)",5.126747
Total Recall (1990),5.084690
Seven (a.k.a. Se7en) (1995),5.067627
"Godfather, The (1972)",5.053508
"Bourne Ultimatum, The (2007)",5.042222


# Surprise 를 이용한 추천 시스템 구축

In [31]:
# 데이터 출처: https://www.kaggle.com/saurav9786/recommender-system-using-amazon-reviews/data
import surprise 

print(surprise.__version__)

1.1.1


In [33]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [49]:
electronics_data = pd.read_csv("ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])
electronics_data.shape

(7824482, 4)

In [58]:
eletronics_data = electronics_data.iloc[:1048576,0:]
electronics_data.head()

,userId,productId,Rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [36]:
electronics_data.shape

(7824482, 4)

In [51]:
electronics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824482 entries, 0 to 7824481
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     object 
 1   productId  object 
 2   Rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


In [87]:
electronics_data.isna().sum()

userId       0
productId    0
Rating       0
timestamp    0
dtype: int64

In [90]:
new_df = electronics_data.groupby('productId').filter(lambda x: x['userId'].count() >= 300)

In [91]:
# 평가가 300번이상 된 상품들을 가져오기

new_df.head()

,userId,productId,Rating,timestamp
183,A1BKC6B7YHYTVV,0972683275,4.0,1405382400
184,AWVFSIB42LHKJ,0972683275,4.0,1405209600
185,A36MQBNADRH8YY,0972683275,5.0,1405641600
186,A3SRXMPLAEZ6DP,0972683275,4.0,1405987200
187,A20XXTXWF2TCPY,0972683275,5.0,1405123200


In [92]:
new_df.shape

(2918593, 4)

In [93]:
new_df.drop(['timestamp'],axis = 1, inplace = True)
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(new_df,reader)

In [94]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

### KNN을 이용 trainset에 파라미터 맞추기

In [96]:
from surprise import KNNWithMeans
# k=5, sim_options = {'name': 'pearson_baseline', 'user_based':False}
algo = KNNWithMeans(k=5, sim_options={'name':'pearson_baseline','user_based':False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [97]:
# prediction 결과의 최초 5개 추출하기
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  875578
prediction 결과의 최초 5개 추출


[Prediction(uid='A1AZWM45BLWB3B', iid='B004ZGN6MY', r_ui=4.0, est=4.137617687584281, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A2AIC68L68L9SI', iid='B008I21EA2', r_ui=1.0, est=4.137617687584281, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A27OZ6NKLINMIG', iid='B0019SHZU0', r_ui=5.0, est=4.137617687584281, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A37X260ISIDBM5', iid='B0043M668G', r_ui=5.0, est=4.179829890643985, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A406Y99RMT21Q', iid='B0015EWMX8', r_ui=3.0, est=4.137617687584281, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [100]:
# Prediction 객체에서 3개의 uid, iid, est 속성 추출하기
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('A1AZWM45BLWB3B', 'B004ZGN6MY', 4.137617687584281),
 ('A2AIC68L68L9SI', 'B008I21EA2', 4.137617687584281),
 ('A27OZ6NKLINMIG', 'B0019SHZU0', 4.137617687584281)]

In [101]:
# RMSE 평가 결과 확인하기
accuracy.rmse(predictions)

RMSE: 1.3090


1.308999000553652